In [25]:
import os
import re
import torch
import numpy as np
import pandas as pd

from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [2]:
device = torch.device('cuda')

# Load Pre-trained GPT2 Model

In [4]:
MODEL_NAME = 'sberbank-ai/rugpt3medium_based_on_gpt2'

model = GPT2LMHeadModel.from_pretrained(MODEL_NAME).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

Downloading: 100%|██████████████████████████████| 674/674 [00:00<00:00, 495kB/s]
Downloading: 100%|█████████████████████████| 1.73G/1.73G [00:27<00:00, 63.7MB/s]
Downloading: 100%|█████████████████████████| 1.61M/1.61M [00:00<00:00, 2.43MB/s]
Downloading: 100%|█████████████████████████| 1.27M/1.27M [00:00<00:00, 1.70MB/s]


# Testing Text Generation

In [27]:
test_text = """
    Жизнь как коробка конфет, никогда не знаешь
"""

In [28]:
inputs = tokenizer(test_text, return_tensors='pt').to(device)
inputs

{'input_ids': tensor([[  203, 23300, 14518,   428, 40544, 45392,    16,  2002,   322,  4515,
           203]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [36]:
generated = model.generate(**inputs, top_p=0.75, 
                           do_sample=True, 
                           min_length=50, 
                           max_length=200,
                           repetition_penalty=4.)
print(*tokenizer.batch_decode(generated))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



    Жизнь как коробка конфет, никогда не знаешь
      когда и куда тебя угостят. И только один раз в жизни я могу попробовать на вкус то что есть у других - это шоколад! С тех пор прошло немало времени... Но все равно каждый день беру его с собой во рту (причем иногда даже вместе со сладостями), хотя он для меня почти недоступен.... Так странно осознавать,что раньше мне казалось нормальным просто проглотить кусочек сладкого продукта(например шоколада) а теперь нет..... Я понимаю почему люди говорят "хочу" но до сихпор так мало кто понимает слово ЛЮБОВЬ!!! Это очень сложное чувство-когда хочется обнять когото или поцеловать того кому действительно нужна твоя любовь!!! В общем надо любить людей всем сердцем!!!! Очень тяжело сейчас жить без этого чувства!!! Вот думаю стоит ли нам продолжать встречаться дальше? Если да если НЕТ? Может быть лучше начать новые отношения..?? Ведь по настоящему любящий человек может дать счастье другим людям..хотя бы одному человеку.Может мы


# Finetuning GPT2 to answer questions like V. Putin

In [38]:
speech_data = pd.read_csv('data/president_speech.csv', encoding='utf8', sep=';')
speech_data.shape

(5526, 6)

In [39]:
speech_data

,Unnamed: 0,url,quote_source,order_in_chat,other_quote,president_quote
0,0,http://www.kremlin.ru/events/president/transcr...,Вопрос,0,"Господин Путин, каково Ваше личное отношение к...","Хотел бы отметить, что у меня хорошие отношени..."
1,1,http://www.kremlin.ru/events/president/transcr...,Вопрос,1,Может быть дружба своего рода?,"Мы знакомы мало. Но я должен сказать, что мне ..."
2,2,http://www.kremlin.ru/events/president/transcr...,Вопрос,2,"Господин Президент, госпожа Политковская – это...","Прежде всего хочу сказать, что это омерзительн..."
3,3,http://www.kremlin.ru/events/president/transcr...,Вопрос,3,Потому что у нее не было возможности выступать...,"Если Вы мне дадите возможность высказаться, то..."
4,4,http://www.kremlin.ru/events/president/transcr...,Вопрос,4,"И поэтому это неправильные публикации, по мнен...","Я еще раз повторяю, будьте так любезны, не пер..."
...,...,...,...,...,...,...
5521,5521,http://www.kremlin.ru/events/president/transcr...,Вопрос,5,Европейская комиссия начала процедуру наложени...,Во время визита господина Конте в Москву эту т...
5522,5522,http://www.kremlin.ru/events/president/transcr...,Вопрос,6,С избранием Владимира Зеленского на пост Прези...,"Да, возможно, если Владимир Зеленский начнёт в..."
5523,5523,http://www.kremlin.ru/events/president/transcr...,Вопрос,7,"У Вас нет настоящих политических противников, ...","Дело не в процентах голосов на выборах, а в эк..."
5524,5524,http://www.kremlin.ru/events/president/transcr...,Вопрос,8,Задумываетесь ли Вы о России после Путина с 20...,Об этом пока рано говорить. Впереди ещё пять л...


# Preparing the data and tokenizer

Adding special tokens to highlight question and answer

In [40]:
data_txt = []
for i in range(speech_data.shape[0]):
    o, p = speech_data.other_quote[i], speech_data.president_quote[i]
    line = f'@@Q_START@@ {o} @@Q_END@@ @@A_START@@ {p} @@A_END@@'
    data_txt.append(line)

In [42]:
data_txt[0]

'@@Q_START@@ Господин Путин, каково Ваше личное отношение к госпоже Федеральному канцлеру? Существуют ли отличия от Ваших отношений с господином Шредером, который с Вами дружил близко? @@Q_END@@ @@A_START@@ Хотел бы отметить, что у меня хорошие отношения с руководством ФРГ еще со времен, когда канцлером был Коль. Несмотря на то что между его преемником и самим господином Колем отношения складывались не самым благообразным образом, у меня продолжались контакты с господином Колем всегда. Мы регулярно встречались с ним в Москве, в Кремле. Это не помешало мне строить хорошие отношения с новым канцлером. У меня очень хорошие деловые отношения и с новым канцлером Меркель. @@A_END@@'

In [43]:
with open('data/data.txt', 'w') as w:
    for ex in data_txt:
        w.write(f'{ex}\n')

In [44]:
! mkdir -p data/new_tokenizer

In [45]:
# Adding the new tokens and saving the new tokenizer

tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_tokens(['@@Q_START@@', '@@Q_END@@', '@@A_START@@', '@@A_END@@'], special_tokens=True)


tokenizer.save_pretrained('data/new_tokenizer')

('data/new_tokenizer/tokenizer_config.json',
 'data/new_tokenizer/special_tokens_map.json',
 'data/new_tokenizer/vocab.json',
 'data/new_tokenizer/merges.txt',
 'data/new_tokenizer/added_tokens.json')

# Training script

Using gradient accumulation to fit into 12 gb GPU memory.

In [ ]:
python run_clm.py \
   --do_train \
   --model_name_or_path "sberbank-ai/rugpt3medium_based_on_gpt2" \
   --tokenizer_name "data/new_tokenizer" \
   --model_type "gpt2" \
   --train_file "data/data.txt" \
   --validation_file "data/data.txt" \
   --cache_dir "data/cache" \
   --output_dir "speech_logs" \
   --per_device_train_batch_size 2 \
   --gradient_accumulation_steps 8 \
   --save_total_limit 3 \
   --save_strategy "steps" \
   --eval_steps 500 \
   --save_steps 500 \
   --preprocessing_num_workers 4 \
   --num_train_epochs 3 \
   --block_size 512

# Load Trained Model

In [59]:
checkpoint_dir = 'speech_logs/checkpoint-500'

model = GPT2LMHeadModel.from_pretrained(checkpoint_dir).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint_dir)

In [60]:
# Memorize special tokens IDs
vocab = tokenizer.get_vocab()
id0, id1, id2, id3 = vocab['@@Q_START@@'], vocab['@@Q_END@@'], vocab['@@A_START@@'], vocab['@@A_END@@']
id0, id1, id2, id3

(50257, 50258, 50259, 50260)

In [71]:
input_text = ['Каковы, на ваш взгляд, перспективы использования паукообразных роботов в сельском хозяйстве?',
              'Каким будет 2023 год?',
              'Оказавшись перед Путиным, что вы ему скажете?']

for t in input_text:

    inputs = tokenizer(f'@@Q_START@@ {t} @@Q_END@@ @@A_START@@', 
                       return_tensors='pt').to(device)
    
    generated = model.generate(**inputs, 
                               top_p=0.7,
                               temperature=0.5,
                               do_sample=True,
                               eos_token_id=id3, 
                               repetition_penalty=4.0, 
                               max_length=100)
    
    generated = tokenizer.batch_decode(generated)[0]
    answer = re.search('@@A_START@@(.*)', generated).group(1)
    answer = re.sub('@@A_END@@', '', answer)
    
    print(f'QUESTION: {t}\nANSWER: {answer}', end='\n')

Setting `pad_token_id` to `eos_token_id`:50260 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50260 for open-end generation.


QUESTION: Каковы, на ваш взгляд, перспективы использования паукообразных роботов в сельском хозяйстве?
ANSWER:   Вы знаете нашу позицию. Мы считаем: и мы не против применения роботизированных систем для решения задач сельского хозяйства – это тоже часть нашей позиции по этому вопросу; но только при условии соблюдения принципов промышленной политики страны с учетом интересов фермеров-производителей сельскохозяйственной продукции либо ее потребителей (это касается как России вообще так же стран СНГ).  


Setting `pad_token_id` to `eos_token_id`:50260 for open-end generation.


QUESTION: Каким будет 2023 год?
ANSWER:   Это сложный вопрос. Я думаю, что мы все должны быть готовы к тому времени и в состоянии решить те задачи развития страны на перспективу – это прежде всего развитие экономики; социальной сферы: здравоохранения (мы сейчас говорим о здравоохранении), образования всех уровней с тем чтобы обеспечить людей качественными услугами по доступным ценам для населения Российской Федерации как минимум до конца этого года или даже раньше).  
QUESTION: Оказавшись перед Путиным, что вы ему скажете?
ANSWER:   Я скажу: «Владимир Владимирович! Вы знаете мою позицию. Она заключается в том числе и во введении визового режима между нашими странами».  
